# SCRIPT EXPLORACIÓN

### Estrategia de Exclusión de Variables

En este notebook, desarrollamos una estrategia para crear una lista de las columnas que fueron excluidas del dataset. Esta lista nos permitió realizar experimentos con diferentes configuraciones de entrenamiento de modelos, probando distintas combinaciones de variables. Sin embargo, algunas de las variables creadas, aunque inicialmente prometedoras, no han proporcionado mejoras significativas en el rendimiento del modelo. Al excluir estas variables, hemos buscado optimizar la capacidad predictiva del modelo y reducir la complejidad innecesaria.

In [ ]:
# Columnas que se excluirán del entrenamiento
excluded_columns = [
    "Temperatura almacén principal (fin)", "Viabilidad inicial cultivo",
    "Turbidez inicio cultivo", "Proporción turbidez", "Orden en el encadenado", "LOTE parental",
    "ID Bioreactor", "Fecha/hora fin", "Fecha/hora inicio", 'Turbidez inicio cultivo',
    "Producto 1", "Producto 2",'Humedad centrifugas (fin)', 'Humedad centrifugas (inicio)', "Indice_humedad_total", "Temperatura almacén principal (inicio)", 
    'Humedad biorreactores (fin)', 'Humedad almacén principal (fin)','ID Centrífuga', "Temperatura almacén principal (fin)",'Humedad almacén principal (inicio)',
    'Temperatura almacén producción (inicio)', 'Temperatura almacén producción (fin)',"periodo_dias","Delta_turbidez",'Proporcion_turbidez','Delta_temp_bio_cen','Temperatura centrifugas (fin)','Indice_humedad_total',
    'Humedad biorreactores (media)','Temperatura almacén producción (media)','Humedad almacén producción (media)','Humedad centrifugas (media)',
    'Temperatura centrifugas (media)','Temperatura biorreactores (inicio)','Turbidez fin cultivo','Proporcion_temp_hum','Temperatura almacén principal (media)'
]

**Proporción turbidez** almacena el cociente entre los valores de Centrifugación 1 turbidez y Centrifugación 2 turbidez.
El valor 1e{-6} (0.000001) se suma al denominador (Centrifugación 2 turbidez) para evitar que este sea igual a cero, ya que dividir por cero genera un error.


In [ ]:
df['Proporcion_turbidez'] = df['Centrifugación 1 turbidez'] / (df['Centrifugación 2 turbidez'] + 1e-6)


**Indice_humedad_total** representa el promedio de tres columnas relacionadas con diferentes niveles de humedad.


In [ ]:
df['Indice_humedad_total'] = (df['Humedad almacén producción (inicio)'] + 
                              df['Humedad biorreactores (inicio)'] + 
                              df['Humedad almacén producción (fin)']) / 3

### Qué hace **periodo_dias** :

1. **Cálculo de diferencia de fechas:**
   - Resta las fechas de las columnas `Fecha/hora fin` y `Fecha/hora inicio` en el DataFrame `df`.
   - Esto da como resultado una serie de objetos `timedelta` (diferencia de tiempo entre dos fechas).

2. **Obtención del número de días:**
   - El atributo `.dt.days` extrae únicamente la diferencia en días de los objetos `timedelta` resultantes.
   - Esta diferencia se guarda como una nueva columna llamada `periodo_dias`.

In [ ]:
df["periodo_dias"] = (df["Fecha/hora fin"] - df["Fecha/hora inicio"]).dt.days

### Qué hace **Delta_turbidez**:

1. **Cálculo de la diferencia de turbidez:**
   - Resta los valores de la columna `Centrifugación 2 turbidez` de los valores en la columna `Centrifugación 1 turbidez`.

2. **Creación de una nueva columna:**
   - La diferencia resultante se almacena en una nueva columna llamada `Delta_turbidez`.

### Interpretación:
La columna `Delta_turbidez` mide el cambio en los niveles de turbidez entre los procesos de **Centrifugación 1** y **Centrifugación 2**. 
- Un valor positivo indica que la turbidez es mayor en la **Centrifugación 1**.
- Un valor negativo indica que la turbidez es mayor en la **Centrifugación 2**.

In [ ]:
df['Delta_turbidez'] = df['Centrifugación 1 turbidez'] - df['Centrifugación 2 turbidez']

### Qué hace **Proporcion_turbidez**:

1. **Cálculo de la proporción de turbidez:**
   - Divide los valores de la columna `Centrifugación 1 turbidez` por los de la columna `Centrifugación 2 turbidez`.

2. **Prevención de divisiones por cero:**
   - Se suma un pequeño valor (\(1 \times 10^{-6}\)) al denominador (`Centrifugación 2 turbidez`) para evitar errores en caso de que el denominador sea igual a cero.

3. **Creación de una nueva columna:**
   - La proporción calculada se almacena en una nueva columna llamada `Proporcion_turbidez`.

In [ ]:
df['Proporcion_turbidez'] = df['Centrifugación 1 turbidez'] / (df['Centrifugación 2 turbidez'] + 1e-6)

### Qué hace **Delta_temp_bio_cen**:

1. **Cálculo de la diferencia de temperaturas:**
   - Resta los valores de la columna `Temperatura centrifugas (fin)` de los valores en la columna `Temperatura biorreactores (fin)`.

2. **Creación de una nueva columna:**
   - La diferencia resultante se almacena en una nueva columna llamada `Delta_temp_bio_cen`.

### Interpretación:
La columna `Delta_temp_bio_cen` representa la diferencia de temperatura al final del proceso entre los **biorreactores** y las **centrífugas**. Un valor positivo indica que la temperatura en los biorreactores es mayor que en las centrífugas, mientras que un valor negativo indica lo contrario.

In [ ]:
df['Delta_temp_bio_cen'] = df['Temperatura biorreactores (fin)'] - df['Temperatura centrifugas (fin)']

### Qué hace Indice_humedad_total:

1. **Cálculo del promedio de humedad:**
   - Suma los valores de las columnas:
     - `Humedad almacén producción (inicio)`
     - `Humedad biorreactores (inicio)`
     - `Humedad almacén producción (fin)`
   - Divide esta suma entre 3 para calcular el promedio.

2. **Creación de una nueva columna:**
   - El promedio calculado se almacena en una nueva columna llamada `Indice_humedad_total`.

### Interpretación:
La columna `Indice_humedad_total` representa un índice promedio que combina los niveles de humedad al inicio y al final del almacén de producción, junto con la humedad inicial de los biorreactores. Esta métrica proporciona una visión agregada de la humedad en el entorno de producción.

In [ ]:
df['Indice_humedad_total'] = (df['Humedad almacén producción (inicio)'] + df['Humedad biorreactores (inicio)'] + df['Humedad almacén producción (fin)']) / 3

### Qué hace **Proporcion_temp_hum**:

1. **Cálculo de la proporción entre temperatura y humedad:**
   - Divide los valores de la columna `Temperatura almacén producción (inicio)` por los de la columna `Humedad almacén producción (inicio)`.

2. **Prevención de divisiones por cero:**
   - Suma un pequeño valor (\(1 \times 10^{-6}\)) al denominador (`Humedad almacén producción (inicio)`) para evitar errores en caso de que el denominador sea igual a cero.

3. **Creación de una nueva columna:**
   - La proporción calculada se almacena en una nueva columna llamada `Proporcion_temp_hum`.

### Interpretación:
La columna `Proporcion_temp_hum` mide la relación entre la temperatura y la humedad al inicio del proceso en el almacén de producción.  
- Un valor alto indica una mayor temperatura relativa en comparación con la humedad.
- Un valor bajo sugiere que la humedad domina en proporción a la temperatura.

In [ ]:
df['Proporcion_temp_hum'] = df['Temperatura almacén producción (inicio)'] / (df['Humedad almacén producción (inicio)'] + 1e-6)

### Explicación sobre el rendimiento de las variables con la media:

En el proceso de construcción del modelo, se utilizaron variables calculadas como la media de varias mediciones, como `Humedad biorreactores (media)`, `Temperatura almacén producción (media)`, `Humedad almacén producción (media)`, `Humedad centrifugas (media)`, `Temperatura centrifugas (media)`, `Temperatura biorreactores (inicio)`, `Turbidez fin cultivo`, `Proporcion_temp_hum`, y `Temperatura almacén principal (media)`. Sin embargo, el rendimiento del modelo no mejoró como se esperaba al incluir estas variables. 

Esto se debe a varias razones posibles:

1. **Pérdida de variabilidad:**  
   Las medias de las variables tienden a suavizar las fluctuaciones y variaciones importantes que podrían contener patrones valiosos para el modelo. Al promediar los datos, se elimina la información sobre cómo estas variables cambian con el tiempo o bajo diferentes condiciones, lo cual puede ser crucial para la predicción.

2. **Falta de diferenciación:**  
   Al ser valores promedio, estas variables no capturan correctamente las diferencias entre los distintos estados o condiciones de las mediciones. El modelo necesita información que pueda diferenciar situaciones o comportamientos específicos, y las medias, al ser más homogéneas, no ofrecen esta capacidad.

3. **Posible correlación con el target limitada:**  
   Las variables promediadas pueden no estar correlacionadas de manera efectiva con el target o el resultado que estamos tratando de predecir, especialmente si hay otros factores más específicos y dinámicos que influyen en el comportamiento de las variables de entrada.

4. **Exceso de generalización:**  
   Usar medias en lugar de valores específicos de cada observación o condición puede haber generalizado demasiado el conjunto de datos, limitando así el poder predictivo del modelo.

En resumen, las medias de las variables no fueron útiles porque, al suavizar la información y perder las variaciones clave, no proporcionaron suficiente información relevante para que el modelo pudiera hacer predicciones precisas.